<a href="https://colab.research.google.com/github/sxg4060/Weight-Sensitivity-Analysis/blob/main/weight_sensitivity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q pyyaml h5py # Required to save models in HDF5 format

In [ ]:
import logging
import tensorflow as tf
import keras
from keras.models import load_model
import numpy as np
import os
import pathlib
import h5py
import re
import sys
import random
from struct import *
from keras import backend as K
from keras.optimizers import RMSprop
from keras.utils.conv_utils import convert_kernel
from keras.optimizers import SGD
from keras.models import Sequential
from numpy import savetxt
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Conv2D
from keras import activations
from keras.utils.conv_utils import convert_kernel
import matplotlib.pyplot as plt
print(tf.version.VERSION)

In [ ]:
import struct

def float_to_bin(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

##bit flip function-flips bit in float value based on position
def weight_bit_flip(weight, flip_position):
    #add 2 to flip position-reflect actual position in number
    #flip_position = flip_position + 2
    #convert weight to hex then binary
    bin_weight = float_to_bin(weight)
    #bin_weight= bin(int.from_bytes(pack('f', weight), byteorder=sys.byteorder))
    print("input bits: " + str(bin_weight))
    bin_weight = list(bin_weight)
    
          
       
    #check to make sure flip is valid position
    if flip_position > (len(bin_weight)):
        print("invalid position for bit flip")
        return

    #flip bit-case for flip at end, middle
    bin_weight_flip=bin_weight
    if bin_weight[flip_position] == str(0):
        print(flip_position)
        bin_weight_flip[flip_position] ='1' 
    else:
        bin_weight_flip[flip_position] ='0'
    
    #join list back into string
    bin_weight_flip="".join(bin_weight_flip)
    
    #convert binary weight back into float-convert to binary to hex to float
    print("output bits: "+bin_weight_flip)
    return bin_weight_flip
    #byte_weight_flip=int(bin_weight_flip,2).to_bytes(4, byteorder=sys.byteorder)
    #weight_flip=unpack('f', byte_weight_flip)
    #print(weight_flip[0])
    #return round(weight_flip[0],10)

def ieee_parser(ieee_binary):
    sign = int(ieee_binary[0])
    if sign == 1:
      sign = -1
    if sign == 0:
      sign = 1
    #print("Sign = ", sign)
    exponent = int(ieee_binary[1:9],2)
    #print("Exponent = ", exponent)
    mantissa = 1
    for i in range(1,24):
      #print(int(ieee_binary[i+8]))
      temp1 = (int(ieee_binary[i+8]))*(2**-i)
      #print(temp1)
      mantissa = mantissa + temp1
    #print("Mantissa = ", mantissa)
    value = sign * ((2)**(exponent-127)) * (mantissa) 
    return value

In [ ]:
op = weight_bit_flip(0.123213,0)
print(ieee_parser(op))

In [ ]:
# Load MNIST dataset 
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
def make_model():
  model = Sequential()
  model.add(Dense(10, input_shape=(784,)))
  model.add(Activation(activations.softmax, input_shape=(10,)))

  model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
  return model
K.set_floatx('float32')
model = make_model()

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=1,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [ ]:
min_wt = x_test.min()

max_wt = x_test.max()

int_bits = int(np.ceil(np.log2(max(abs(min_wt),abs(max_wt)))))

frac_bits = 3-int_bits

quant_weight = np.round((x_test)*(2**frac_bits))

recovered_weight = quant_weight/(2**frac_bits)

min_wt_y = y_test.min()

max_wt_y = y_test.max()

int_bits_y = int(np.ceil(np.log2(max(abs(min_wt_y),abs(max_wt_y)))))

frac_bits_y = 3-int_bits_y

quant_weight_y = np.round((y_test)*(2**frac_bits_y))

recovered_weight_y = quant_weight_y/(2**frac_bits_y)
print('Original weights: ', model.get_weights())
print('Quantized weights:', quant_weight[0])
print('Recovered weights: ', recovered_weight[0])

scores = model.evaluate(x_test, y_test, verbose=0)


print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# save model and architecture to single file
model.save("model.h5")
model.save_weights("trained_weights.h5")
print("Saved model to disk")

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

print("====Model Weights====")
print(model.get_weights()[0])
print("====Model Predictions===")
print(model.predict(x_test, batch_size=1))

In [ ]:
# Layer Sensitivity Analysis

amount = 0.1
# Create new model_prime with trained weights
model.load_weights('trained_weights.h5')
layer_i = model.layers[0]
#print(layer_i.get_weights()[0])

# Variables to traverse array

a = layer_i.get_weights()[0] * 1
b = layer_i.get_weights()[1] * 1

model.layers[0].set_weights([a,b])
#print(model.layers[0].get_weights()[0])


model.save_weights('modified_weight.h5')

In [ ]:
# RH ALL
orig_array=model.layers[0].get_weights()
orig_weight=orig_array[0][0, 0]

##enter loop to modify number with bit flip
##pull initial value for weight
orig_array=model.layers[0].get_weights()
orig_weight=orig_array[0][0,0]
##initial weight array
bit_flip_array=orig_array
x_array = []
y_array = []
acc_array = []
for x in range(783, 0, -1):
  for y in range(9, 0, -1):
      print("Weight to be modified = " + str(orig_array[0][x,y]))
      bit_flip_array[0][x,y] = ieee_parser(weight_bit_flip(orig_array[0][x,y], 0))
      print("new weight: " +str(bit_flip_array[0][x,y]))
      model.layers[0].set_weights(bit_flip_array)
      model.save_weights('modified_weight.h5')
      load_model = make_model()
      load_model.load_weights('modified_weight.h5')
      updated_score = load_model.evaluate(x_test,  y_test, verbose=2)
      print("%s: %.2f%%" % (load_model.metrics_names[1], updated_score[1]*100))
      print('Test loss:', updated_score[0])
      print('Test accuracy:', updated_score[1])
      x_array.append(x)
      y_array.append(y)
      acc_array.append(updated_score[1])
      bit_flip_array[0][x,y] = -1 * orig_array[0][x,y]
      model.layers[0].set_weights(bit_flip_array)
      model.save_weights('modified_weight.h5')

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
acc32_array = acc_array
x32_array = x_array
y32_array = y_array
# Data for a three-dimensional line
zline = acc32_array
xline = x32_array
yline = y32_array
ax.scatter3D(xline, yline, zline, c=zline, cmap='Greens');
print(acc_array)
print(max(acc_array))
print(min(acc_array))
plt.show()